In [7]:
import pandas as pd
train = pd.read_csv("training.csv",header=0)
test = pd.read_csv("test.csv",header=0)
valid = pd.read_csv("validation.csv",header=0)


In [8]:
#using the keras

from nltk.tokenize import word_tokenize
def token(content):
    return word_tokenize(content,"english")

#remove stop words
from nltk.corpus import stopwords
final_words=[]
def rm_stopwords(words):
   final_words = [word for word in words if word not in stopwords.words("english")]
   return final_words

# stemming + lemmatization
from nltk.stem import SnowballStemmer 
snowball = SnowballStemmer(language='english')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def many_stem_lemma(list_of_words):
    new_list =[]
    for word in list_of_words:
      stem =  snowball.stem(word)
      lemma = lemmatizer.lemmatize(stem,pos='v')
      new_list.append(lemma)
    return new_list

#combining all
import re
def transf(x):
    lower_content = x.lower()
    punct_content = re.sub(r'[^\w\s]', '', lower_content)

    L = many_stem_lemma(rm_stopwords(token(punct_content)))
    return L

train['text'] = train['text'].apply(lambda x: transf(x))
test['text'] = test['text'].apply(lambda x: transf(x))
valid['text'] = valid['text'].apply(lambda x: transf(x))

x_train = train['text']
y_train = train['label']
x_test = test['text']
y_test = test['label']
x_valid = valid['text']
y_valid = valid['label']

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

total_reviews=[]
total_reviews.extend(x_train)
total_reviews.extend(x_test)
total_reviews.extend(x_valid)

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(total_reviews)

max_length = max([len(s) for s in total_reviews])
vocab_size = len(tokenizer_obj.word_index) + 1

x_train_tokens = tokenizer_obj.texts_to_sequences(x_train)
x_test_tokens = tokenizer_obj.texts_to_sequences(x_test)
x_valid_tokens = tokenizer_obj.texts_to_sequences(x_valid)

x_train_pad = pad_sequences(x_train_tokens, maxlen=max_length, padding='post')
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_length, padding='post')
x_valid_pad = pad_sequences(x_valid_tokens, maxlen=max_length, padding='post')


In [9]:
#using keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional


model4 = Sequential()
model4.add(Embedding(vocab_size, 32, input_length=max_length))
model4.add(LSTM(32, return_sequences=True, recurrent_dropout=0.2))
model4.add(LSTM(64, return_sequences=True, recurrent_dropout=0.2))
model4.add(LSTM(128, recurrent_dropout=0.2))
model4.add(Dense(6, activation='softmax'))

model4.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'], run_eagerly=True)

In [10]:
import tensorflow as tf
tf.keras.backend.clear_session()

from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train, num_classes=6)
y_test_one_hot = to_categorical(y_test, num_classes=6)

model4.fit(x_train_pad, y_train_one_hot, batch_size=10, epochs=10, validation_data=(x_test_pad, y_test_one_hot))


Epoch 1/10
1600/1600 [==============================] - 5083s 3s/step - loss: 1.5857 - accuracy: 0.3314 - val_loss: 1.5592 - val_accuracy: 0.3475
Epoch 2/10
1600/1600 [==============================] - 2283s 1s/step - loss: 1.5772 - accuracy: 0.3334 - val_loss: 1.5591 - val_accuracy: 0.3475
Epoch 3/10
 140/1600 [=>............................] - ETA: 29:19 - loss: 1.6126 - accuracy: 0.3229

KeyboardInterrupt: 